In [6]:
import numpy as np
import pandas as pd
import datetime
import os

from glob import glob
from tqdm.notebook import tqdm

import hashlib
from tqdm import tqdm_notebook as tqdm

from rdflib import Namespace, Literal, URIRef
from rdflib.graph import Graph, ConjunctiveGraph
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD

In [7]:
def check_time() : 
    now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    return now

In [8]:
# 기본 경로
os.chdir(r'C:\Users\haklaekim\Desktop\address-data-rdf') # 이 폴더 경로만 수정해주면 됨

today = datetime.date.today().strftime('%Y-%m-%d')
now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print('현재 시각 :', now)

현재 시각 : 2022-08-15 17:51:59


# RDF 변환



In [9]:
# namespace 

kord_def ="http://data.kisti.re.kr/administration/road-name-address/def/" #클래스와 속성
kord_id ="http://data.kisti.re.kr/administration/road-name-address/id/" #인스턴스
KORD = Namespace("http://data.kisti.re.kr/administration/road-name-address/def/") #네임스페이스

koro_def ="http://data.kisti.re.kr/administration/road-name/def/" #클래스와 속성
koro_id ="http://data.kisti.re.kr/administration/road-name/id/" #인스턴스
KORO = Namespace("http://data.kisti.re.kr/administration/road-name/def/") #네임스페이스

koad_def ="http://data.kisti.re.kr/administration/administrative-division/def/" #클래스와 속성
koad_id ="http://data.kisti.re.kr/administration/administrative-division/id/" #인스턴스
KOAD = Namespace("http://data.kisti.re.kr/administration/administrative-division/def/") #네임스페이스

kopc_def ="http://data.kisti.re.kr/administration/postal-code/def/" #클래스와 속성
kopc_id ="http://data.kisti.re.kr/administration/postal-code/id/" #인스턴스
KOPC = Namespace("http://data.kisti.re.kr/administration/postal-code/def/") #네임스페이스


# 함수

def cell(store, s, p, df_col, datatype = None, lang = None):
    if df_col != None:
        store.add((s, p, Literal(df_col, datatype=datatype, lang = lang)))
        
def uri(store, s, p, df_col, objClass = None, objURI = kord_id) :
    if df_col != None :
        obj = URIRef(objURI+df_col) #uri체계 다르면 objURI 매개변수 변경
        store.add((s, p, obj))
        if objClass != None :
            store.add((obj, RDF.type, objClass))


### 주소정보

In [10]:
print(check_time())

list_juso = glob('id-data/juso_*')
len_juso_df = pd.DataFrame()


for filenm in tqdm(list_juso) : 
    
    juso = pd.read_csv(filenm, dtype = str, encoding = 'utf-8')
    juso = juso.replace({np.nan : None})

    # id 값 hash md5 변환
    juso['roadnamead_id'] = juso['roadnamead_id'].apply(lambda x : hashlib.md5(str(x).encode('utf-8')).hexdigest() )

    print(filenm, '전처리 끝 ------------------------------------------------------')

    store = Graph() #트리플스토어 생성
    store.bind("kord", KORD)
    store.bind("koad", KOAD)
    store.bind("koro", KORO)


    for idx, row in juso.iterrows(): #행마다 반복
        # base uri
        sub_rna = URIRef(kord_id + str(row['roadnamead_id']))
        
        # type
        store.add((sub_rna, RDF.type, KORD.RoadNameAddress))
        
        # property
        cell(store, sub_rna, KORD.isUnderground, Literal(row['isUnderground'], datatype=XSD.boolean))
        cell(store, sub_rna, KORD.buildingMainNumber, Literal(row['buildingMainNumber']))
        cell(store, sub_rna, KORD.buildingSubNumber, Literal(row['buildingSubNumber']))
        cell(store, sub_rna, KORD.isDetailedAddress, Literal(row['isDetailedAddress'], datatype=XSD.boolean))
        cell(store, sub_rna, KORD.revisedCode, row['revisedCode'])
        
        uri(store, sub_rna, KORO.roadNameCode, row['roadNameCode'], objClass = None, objURI = koro_id)
        uri(store, sub_rna, KORO.roadNameCodeDetail, str(row['roadNameCode'] + row['emdSerialNumber']), objClass = None, objURI = koro_id)
        
        if row['isDetailedAddress'] == '0' :
            cell(store, sub_rna, KORD.buildingManagementNumber, Literal(row['buildingManagementNumber']))
        else :
            continue 
    
    # result download
    download_nm = filenm.split('.csv')[0][8:].split('_')[-1]
    store.serialize(destination = f'graph-data/juso/juso_{download_nm}_{today}.ttl') 
    
    # 수량 집계
    len_juso = [download_nm, len(store)]
    len_juso_df = len_juso_df.append(pd.Series(len_juso), ignore_index = True)
    print(download_nm, len(store), '----', check_time())

# 수량 count
len_juso_df.columns = ['지역명', 'triple_count']
len_juso_df.to_excel('data_count/juso_data_len.xlsx', index = False)

2022-08-15 17:53:56


C:\Users\haklaekim\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/17 [00:00<?, ?it/s]

id-data\juso_id_busan.csv 전처리 끝 ------------------------------------------------------
busan 2982725 ---- 2022-08-15 17:58:39
id-data\juso_id_chungbuk.csv 전처리 끝 ------------------------------------------------------
chungbuk 3231085 ---- 2022-08-15 18:03:50
id-data\juso_id_chungnam.csv 전처리 끝 ------------------------------------------------------
chungnam 4570477 ---- 2022-08-15 18:11:08
id-data\juso_id_daegu.csv 전처리 끝 ------------------------------------------------------
daegu 2606952 ---- 2022-08-15 18:15:35
id-data\juso_id_daejeon.csv 전처리 끝 ------------------------------------------------------
daejeon 1676182 ---- 2022-08-15 18:18:19
id-data\juso_id_gangwon.csv 전처리 끝 ------------------------------------------------------
gangwon 3220921 ---- 2022-08-15 18:23:23
id-data\juso_id_gwangju.csv 전처리 끝 ------------------------------------------------------
gwangju 1464648 ---- 2022-08-15 18:25:47
id-data\juso_id_gyeongbuk.csv 전처리 끝 ------------------------------------------------------
gye

### 부가정보

In [11]:
print(check_time())

list_add = glob('id-data/add_*')
len_add_df = pd.DataFrame()

for filenm in tqdm(list_add) :

    add = pd.read_csv(filenm, dtype = str, encoding = 'utf-8')
    add = add.replace({np.nan : None})

    # 지번id 만들기
    add['jibun_id'] = add['jibunad_id'].apply(lambda a : str(a)[:19])

    # id 값 hash md5 변환
    add['roadnamead_id'] = add['roadnamead_id'].apply(lambda x : hashlib.md5(str(x).encode('utf-8')).hexdigest() )
    add['jibunad_id'] = add['jibunad_id'].apply(lambda x : hashlib.md5(str(x).encode('utf-8')).hexdigest() )
    add['jibun_id'] = add['jibun_id'].apply(lambda x : hashlib.md5(str(x).encode('utf-8')).hexdigest() )
    
    # 행정동_읍면동
    add['town'] = add.apply(lambda x : x.divisionCodeAD[:8] + '00' if str(x['emdName_AD'])[-1:] == '읍' else None, axis = 1)
    add['township'] = add.apply(lambda x : x.divisionCodeAD[:8] + '00' if str(x['emdName_AD'])[-1:] == '면' else None, axis = 1)
    add['neighborhood'] = add.apply(lambda x : None if (str(x['emdName_AD']).endswith(('읍', '면', 'e'))) else str(x.divisionCodeAD)[:8] + '00', axis = 1)
    print(filenm, '전처리 끝 ------------------------------------------------------')

    # 그래프변환
    store = Graph() #트리플스토어 생성
    store.bind("kord", KORD) # 앞에 prefix 추가
    store.bind("koad", KOAD)
    store.bind("koro", KORO)
    # store.bind("kopc", KOPC) 

    for idx, row in add.iterrows(): #행마다 반복
        # base
        sub_rna = URIRef(kord_id + row['roadnamead_id'])
        sub_lna = URIRef(kord_id + row['jibunad_id'])
        sub_land = URIRef(kord_id + row['jibun_id'])
        
        # type
        store.add((sub_rna, RDF.type, KORD.RoadNameAddress))
        store.add((sub_lna, RDF.type, KORD.LandLotNumberAddress))
        store.add((sub_land, RDF.type, KORD.LandLotNumber))
        
        #postalCode
        uri(store, sub_rna, KOPC.postalCode, row['postalCode'], objURI = kopc_id)
        uri(store, sub_lna, KOPC.postalCode, row['postalCode'], objURI = kopc_id)
        
        #buildingName
        cell(store, sub_rna, KORD.buildingNameOfficial, row['buildingNameOfficial'], lang = "ko")
        cell(store, sub_lna, KORD.buildingNameOfficial, row['buildingNameOfficial'], lang = "ko")

        cell(store, sub_rna, KORD.buildingNameLocal, row['buildingNameLocal'], lang = "ko")
        cell(store, sub_lna, KORD.buildingNameLocal, row['buildingNameLocal'], lang = "ko")
        
        # isMultiFamilyHousing
        cell(store, sub_rna, KORD.isMultiFamilyHousing, row['isMultiFamilyHousing'], datatype = XSD.boolean)        
        
        # koad_AD
        uri(store, sub_rna, KOAD.town, row['town'], objURI = koad_id)
        uri(store, sub_rna, KOAD.township, row['township'], objURI = koad_id)
        uri(store, sub_rna, KOAD.neighborhood, row['neighborhood'], objURI = koad_id)
        
        uri(store, sub_land, KOAD.town, row['town'], objURI = koad_id)
        uri(store, sub_land, KOAD.township, row['township'], objURI = koad_id)
        uri(store, sub_land, KOAD.neighborhood, row['neighborhood'], objURI = koad_id)
        
        
    # result download   
    download_nm = filenm.split('.csv')[0][8:].split('_')[-1]
    store.serialize(destination = f'graph-data/add/add_{download_nm}_{today}.ttl')

    # 수량 집계
    len_triple = len(store)
    len_add = [download_nm, len_triple]
    len_add_df = len_add_df.append(pd.Series(len_add), ignore_index = True)
    print(download_nm, len_triple, '----', check_time())

# 수량 count
len_add_df.columns = ['지역명', 'triple_count']
len_add_df.to_excel('data_count/add_data_len.xlsx', index = False)

2022-08-15 19:38:19


C:\Users\haklaekim\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/17 [00:00<?, ?it/s]

id-data\add_id_busan.csv 전처리 끝 ------------------------------------------------------
busan 3023543 ---- 2022-08-15 19:43:52
id-data\add_id_chungbuk.csv 전처리 끝 ------------------------------------------------------
chungbuk 3210439 ---- 2022-08-15 19:50:02
id-data\add_id_chungnam.csv 전처리 끝 ------------------------------------------------------
chungnam 4541004 ---- 2022-08-15 19:58:44
id-data\add_id_daegu.csv 전처리 끝 ------------------------------------------------------
daegu 2548979 ---- 2022-08-15 20:03:45
id-data\add_id_daejeon.csv 전처리 끝 ------------------------------------------------------
daejeon 1625566 ---- 2022-08-15 20:06:54
id-data\add_id_gangwon.csv 전처리 끝 ------------------------------------------------------
gangwon 3153383 ---- 2022-08-15 20:12:44
id-data\add_id_gwangju.csv 전처리 끝 ------------------------------------------------------
gwangju 1439288 ---- 2022-08-15 20:15:31
id-data\add_id_gyeongbuk.csv 전처리 끝 ------------------------------------------------------
gyeongbuk 6

### 지번정보

In [12]:
print(check_time())

list_jibun = glob('id-data/jibun_*')
len_jibun_df = pd.DataFrame()


for filenm in tqdm(list_jibun) :

    land = pd.read_csv(filenm, dtype = str, encoding = 'utf-8')
    land = land.replace({np.nan : None})

    # 데이터 정제
    # id 값 hash md5 변환
    land['roadnamead_id'] = land['roadnamead_id'].apply(lambda x : hashlib.md5(str(x).encode('utf-8')).hexdigest() if x != None else x )
    land['jibunad_id'] = land['jibunad_id'].apply(lambda x : hashlib.md5(str(x).encode('utf-8')).hexdigest() if x != None else x )
    land['jibun_id'] = land['jibun_id'].apply(lambda x : hashlib.md5(str(x).encode('utf-8')).hexdigest() )

    # 시 도 시 군 구 읍 면 동
    # 시도 koad:city koad:province 
    land['city_sido'] = land.apply(lambda x : x.divisionCode[:2] + '00000000' if x['sidoName'][-1:] == '시' else None, axis = 1)
    land['province_sido'] = land.apply(lambda x : x.divisionCode[:2] + '00000000' if x['sidoName'][-1:] == '도' else None, axis = 1)

    # 시군구 koad:city koad:district koad:county 
    land['city_sigungu'] = land.apply(lambda x : x.divisionCode[:5] + '00000' if str(x['sigunguName'])[-1:] == '시' else None, axis = 1)
    land['district_sigungu'] = land.apply(lambda x : x.divisionCode[:5] + '00000' if str(x['sigunguName'])[-1:] == '구' else None, axis = 1)
    land['county_sigungu'] = land.apply(lambda x : x.divisionCode[:5] + '00000' if str(x['sigunguName'])[-1:] == '군' else None, axis = 1)

    # 일반구의 시
    land['city_normalcity'] = land.apply(lambda x : x.divisionCode[:4] + '000000' if len(str(x['sigunguName'])) > 5 else None, axis = 1)

    # 읍면동
    land['town'] = land.apply(lambda x : x.divisionCode[:8] + '00' if str(x['emdName'])[-1:] == '읍' else None, axis = 1)
    land['township'] = land.apply(lambda x : x.divisionCode[:8] + '00' if str(x['emdName'])[-1:] == '면' else None, axis = 1)
    land['neighborhood'] = land.apply(lambda x : None if (str(x['emdName']).endswith(('읍', '면', 'e'))) else str(x.divisionCode)[:8] + '00', axis = 1)

    # 리
    land['village'] = land.apply(lambda x : None if (str(x['liName']) == 'None') else x.divisionCode, axis = 1)

    print(filenm, '전처리 끝 ------------------------------------------------------')
    # rdflib   
    store = Graph() #트리플스토어 생성
    store.bind("kord", KORD) # 앞에 prefix 추가
    store.bind("koad", KOAD)
    store.bind("koro", KORO)

    for idx, row in land.iterrows(): 
        # base
        sub_rna = URIRef(kord_id + row['roadnamead_id'])
        sub_lna = URIRef(kord_id + row['jibunad_id'])
        sub_land = URIRef(kord_id + row['jibun_id'])
        
        # type
        store.add((sub_rna, RDF.type, KORD.RoadNameAddress))
        store.add((sub_lna, RDF.type, KORD.LandLotNumberAddress))
        store.add((sub_land, RDF.type, KORD.LandLotNumber))
        
        # represenat-related
        if row['isRepresentative'] == '1' : 
            uri(store, sub_lna, KORD.represnetativeLandLotNumber, row['jibun_id'], KORD.LandLotNumber)
        else :
            uri(store, sub_lna, KORD.relatedLandLotNumber, row['jibun_id'], KORD.LandLotNumber)
        
        # landLotNumber
        cell(store, sub_land, KORD.landLotMainNumber, row['landLotMainNumber'])
        cell(store, sub_land, KORD.landLotSubNumber, row['landLotSubNumber'])

        # isMountain
        cell(store, sub_land, KORD.isMountain, row['isMountain'], XSD.boolean)
        
        # roadnameaddress - landlotnumberaddress
        uri(store, sub_rna, KORD.isAddressLOf, row['jibunad_id'], KORD.LandLotNumberAddress, objURI = kord_id)
        uri(store, sub_lna, KORD.isAddressROf, row['roadnamead_id'], KORD.RoadNameAddress, objURI = kord_id)
        
        
        # koad_AD : rna
        uri(store, sub_rna, KOAD.city, row['city_sido'], objURI = koad_id)
        uri(store, sub_rna, KOAD.province, row['province_sido'], objURI = koad_id)
        uri(store, sub_rna, KOAD.city, row['city_sigungu'], objURI = koad_id)
        uri(store, sub_rna, KOAD.district, row['district_sigungu'], objURI = koad_id)
        uri(store, sub_rna, KOAD.county, row['county_sigungu'], objURI = koad_id)
        uri(store, sub_rna, KOAD.city, row['city_normalcity'], objURI = koad_id)
        uri(store, sub_rna, KOAD.town, row['town'], objURI = koad_id)
        uri(store, sub_rna, KOAD.township, row['township'], objURI = koad_id)
        uri(store, sub_rna, KOAD.neighborhood, row['neighborhood'], objURI = koad_id)
        uri(store, sub_rna, KOAD.village, row['village'], objURI = koad_id)

        # koad_AD : land
        uri(store, sub_land, KOAD.city, row['city_sido'], objURI = koad_id)
        uri(store, sub_land, KOAD.province, row['province_sido'], objURI = koad_id)
        uri(store, sub_land, KOAD.city, row['city_sigungu'], objURI = koad_id)
        uri(store, sub_land, KOAD.district, row['district_sigungu'], objURI = koad_id)
        uri(store, sub_land, KOAD.county, row['county_sigungu'], objURI = koad_id)
        uri(store, sub_land, KOAD.city, row['city_normalcity'], objURI = koad_id)
        uri(store, sub_land, KOAD.town, row['town'], objURI = koad_id)
        uri(store, sub_land, KOAD.township, row['township'], objURI = koad_id)
        uri(store, sub_land, KOAD.neighborhood, row['neighborhood'], objURI = koad_id)
        uri(store, sub_land, KOAD.village, row['village'], objURI = koad_id)
        

    # result download
    download_nm = filenm.split('.csv')[0][8:].split('_')[-1]
    store.serialize(destination = f'graph-data/jibun/jibun_{download_nm}_{today}.ttl')

    # 수량집계
    len_triple = len(store)
    len_jibun = [download_nm, len_triple]
    len_jibun_df = len_jibun_df.append(pd.Series(len_jibun), ignore_index = True)
    print(download_nm, len_triple, '----', check_time())
    
# 수량 count
len_jibun_df.columns = ['지역명', 'triple_count']
len_jibun_df.to_excel('data_count/jibun_data_len.xlsx', index = False)

# 테스트용으로 실행해둠

2022-08-15 21:39:19


C:\Users\haklaekim\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/17 [00:00<?, ?it/s]

id-data\jibun_id_busan.csv 전처리 끝 ------------------------------------------------------
busan 5439761 ---- 2022-08-15 21:50:12
id-data\jibun_id_chungbuk.csv 전처리 끝 ------------------------------------------------------
chungbuk 6839793 ---- 2022-08-15 22:03:07
id-data\jibun_id_chungnam.csv 전처리 끝 ------------------------------------------------------
chungnam 10225239 ---- 2022-08-15 22:22:54
id-data\jibun_id_daegu.csv 전처리 끝 ------------------------------------------------------
daegu 4501103 ---- 2022-08-15 22:33:00
id-data\jibun_id_daejeon.csv 전처리 끝 ------------------------------------------------------
daejeon 2686891 ---- 2022-08-15 22:38:34
id-data\jibun_id_gangwon.csv 전처리 끝 ------------------------------------------------------
gangwon 7316467 ---- 2022-08-15 22:52:33
id-data\jibun_id_gwangju.csv 전처리 끝 ------------------------------------------------------
gwangju 2527240 ---- 2022-08-15 22:58:14
id-data\jibun_id_gyeongbuk.csv 전처리 끝 -------------------------------------------------

### 상세주소

In [13]:
print(check_time())

list_detail = glob('id-data/detail_*')
len_detail_df = pd.DataFrame()

for filenm in tqdm(list_detail) :

    detail = pd.read_csv(filenm, dtype = str, encoding = 'utf-8')
    detail = detail.replace({np.nan : None})

    # 데이터 정제
    # id 값 hash md5 변환
    detail['roadnamead_id'] = detail['roadnamead_id'].apply(lambda x : hashlib.md5(str(x).encode('utf-8')).hexdigest() )
    detail['jibunad_id'] = detail['jibunad_id'].apply(lambda x : hashlib.md5(str(x).encode('utf-8')).hexdigest() )

    print(filenm, '전처리 끝 ------------------------------------------------------')

    # rdf lib
    store = Graph() #트리플스토어 생성
    store.bind("kord", KORD) # 앞에 prefix 추가
    store.bind("koad", KOAD)
    store.bind("koro", KORO)

    for idx, row in detail.iterrows():
        # base
        sub_rna = URIRef(kord_id + row['roadnamead_id'])
        sub_lna = URIRef(kord_id + row['jibunad_id'])
        
        # type
        store.add((sub_rna, RDF.type, KORD.RoadNameAddress))
        store.add((sub_lna, RDF.type, KORD.LandLotNumberAddress))

        # RNA 
        cell(store, sub_rna, KORD.blockSerialNumber, row['blockSerialNumber'])
        cell(store, sub_rna, KORD.floorSerialNumber, row['floorSerialNumber'])
        cell(store, sub_rna, KORD.unitSuffixSerialNumber, row['unitSuffixSerialNumber'])
        cell(store, sub_rna, KORD.blockName, row['blockName'])
        cell(store, sub_rna, KORD.floorName, row['floorName'])
        cell(store, sub_rna, KORD.floorName, row['floorName'])
        cell(store, sub_rna, KORD.unitName, row['unitName'])
        cell(store, sub_rna, KORD.unitSuffixName, row['unitSuffixName']) 
        cell(store, sub_rna, KORD.isUndergroundFloor, row['isUndergroundFloor'], datatype = XSD.boolean)
        cell(store, sub_rna, KORD.buildingManagementNumber, row['buildingManagementNumber'])
        

        # LNA
        cell(store, sub_lna, KORD.blockSerialNumber, row['blockSerialNumber'])
        cell(store, sub_lna, KORD.floorSerialNumber, row['floorSerialNumber'])
        cell(store, sub_lna, KORD.unitSuffixSerialNumber, row['unitSuffixSerialNumber'])
        cell(store, sub_lna, KORD.blockName, row['blockName'])
        cell(store, sub_lna, KORD.floorName, row['floorName'])
        cell(store, sub_lna, KORD.floorName, row['floorName'])
        cell(store, sub_lna, KORD.unitName, row['unitName'])
        cell(store, sub_lna, KORD.unitSuffixName, row['unitSuffixName'])
        cell(store, sub_lna, KORD.isUndergroundFloor, row['isUndergroundFloor'], datatype = XSD.boolean)
        
    # result download 
    download_nm = filenm.split('.csv')[0][8:].split('_')[-1]
    store.serialize(destination = f'graph-data/detail/detail_{download_nm}_{today}.ttl') 

    # 수량집계
    len_triple = len(store)
    len_detail = [download_nm, len_triple]
    len_detail_df = len_detail_df.append(pd.Series(len_detail), ignore_index = True)
    print(download_nm, len_triple, '----', check_time())

# 수량 count
len_detail_df.columns = ['지역명', 'triple_count']
len_detail_df.to_excel('data_count/detail_data_len.xlsx', index = False)

2022-08-16 02:09:59


C:\Users\haklaekim\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/17 [00:00<?, ?it/s]

id-data\detail_id_busan.csv 전처리 끝 ------------------------------------------------------
busan 1486331 ---- 2022-08-16 02:12:15
id-data\detail_id_chungbuk.csv 전처리 끝 ------------------------------------------------------
chungbuk 1325234 ---- 2022-08-16 02:14:13
id-data\detail_id_chungnam.csv 전처리 끝 ------------------------------------------------------
chungnam 1518409 ---- 2022-08-16 02:16:27
id-data\detail_id_daegu.csv 전처리 끝 ------------------------------------------------------
daegu 2199879 ---- 2022-08-16 02:19:43
id-data\detail_id_daejeon.csv 전처리 끝 ------------------------------------------------------
daejeon 1788707 ---- 2022-08-16 02:22:25
id-data\detail_id_gangwon.csv 전처리 끝 ------------------------------------------------------
gangwon 847385 ---- 2022-08-16 02:23:43
id-data\detail_id_gwangju.csv 전처리 끝 ------------------------------------------------------
gwangju 1124844 ---- 2022-08-16 02:25:22
id-data\detail_id_gyeongbuk.csv 전처리 끝 -------------------------------------------